In [1]:
import gc
import os
from pathlib import Path
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

In [0]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
!ls

building_metadata.csv	   test.csv	  weather_test.csv
building_metadata.feather  test.csv.zip   weather_test.csv.zip
sample_data		   test.feather   weather_test.feather
sample_submission.csv	   train.csv	  weather_train.csv
sample_submission.csv.zip  train.csv.zip  weather_train.csv.zip
sample_submission.feather  train.feather  weather_train.feather


In [0]:
train_df = pd.read_csv('train.csv')
weather_train_df = pd.read_csv('weather_train.csv')
building_meta_df = pd.read_csv('building_metadata.csv')

In [22]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [0]:
root = './'
train_df = pd.read_csv(os.path.join(root, 'train.csv'))
weather_train_df = pd.read_csv(os.path.join(root, 'weather_train.csv'))
test_df = pd.read_csv(os.path.join(root, 'test.csv'))
weather_test_df = pd.read_csv(os.path.join(root, 'weather_test.csv'))
building_meta_df = pd.read_csv(os.path.join(root, 'building_metadata.csv'))
sample_submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))

In [28]:
gc.collect()

387

In [29]:
%%time

train_df.to_feather('train.feather')
test_df.to_feather('test.feather')
weather_train_df.to_feather('weather_train.feather')
weather_test_df.to_feather('weather_test.feather')
building_meta_df.to_feather('building_metadata.feather')
sample_submission.to_feather('sample_submission.feather')

CPU times: user 2.63 s, sys: 5.09 s, total: 7.73 s
Wall time: 16.6 s


In [4]:
%%time

train_df = pd.read_feather('train.feather')
weather_train_df = pd.read_feather('weather_train.feather')
building_meta_df = pd.read_feather('building_metadata.feather')

CPU times: user 677 ms, sys: 298 ms, total: 975 ms
Wall time: 831 ms


In [5]:
type(train_df)

pandas.core.frame.DataFrame

In [0]:
# train_df['date'] = train_df['timestamp'].dt.date
train_df['date'] = pd.to_datetime(train_df.timestamp, format='%Y-%m-%d %H:%M')
train_df['meter_reading_log1p'] = np.log1p(train_df['meter_reading'])

In [0]:
def plot_date_usage(train_df, meter=0, building_id=0):
    train_temp_df = train_df[train_df['meter'] == meter]
    train_temp_df = train_temp_df[train_temp_df['building_id'] == building_id]    
    train_temp_df_meter = train_temp_df.groupby('date')['meter_reading_log1p'].sum()
    train_temp_df_meter = train_temp_df_meter.to_frame().reset_index()
    fig = px.line(train_temp_df_meter, x='date', y='meter_reading_log1p')
    fig.show()

In [24]:
building_meta_df[building_meta_df.site_id == 0]

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
...,...,...,...,...,...,...
100,0,100,Lodging/residential,24456,1968.0,NaN
101,0,101,Office,18860,1986.0,NaN
102,0,102,Office,15876,1983.0,NaN
103,0,103,Education,21657,2016.0,NaN


In [0]:
train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [0]:
debug=False

In [0]:
def preprocess(df):
    df['date'] = pd.to_datetime(df.timestamp, format='%Y-%m-%d %H:%M')
    df["hour"] = df["date"].dt.hour
#     df["day"] = df["timestamp"].dt.day
    df["weekend"] = df["date"].dt.weekday
    df["month"] = df["date"].dt.month
    df["dayofweek"] = df["date"].dt.dayofweek

#     hour_rad = df["hour"].values / 24. * 2 * np.pi
#     df["hour_sin"] = np.sin(hour_rad)
#     df["hour_cos"] = np.cos(hour_rad)

In [0]:
preprocess(train_df)

In [0]:
df_group = train_df.groupby('building_id')['meter_reading_log1p']
building_mean = df_group.mean().astype(np.float16)
building_median = df_group.median().astype(np.float16)
building_min = df_group.min().astype(np.float16)
building_max = df_group.max().astype(np.float16)
building_std = df_group.std().astype(np.float16)

train_df['building_mean'] = train_df['building_id'].map(building_mean)
train_df['building_median'] = train_df['building_id'].map(building_median)
train_df['building_min'] = train_df['building_id'].map(building_min)
train_df['building_max'] = train_df['building_id'].map(building_max)
train_df['building_std'] = train_df['building_id'].map(building_std)

In [42]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [46]:
weather_train_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature           0
cloud_coverage        17228
dew_temperature           0
precip_depth_1_hr     26273
sea_level_pressure     8755
wind_direction            0
wind_speed                0
dtype: int64

In [0]:
weather_train_df = weather_train_df.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [0]:
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [0]:
add_lag_feature(weather_train_df, window=3)
add_lag_feature(weather_train_df, window=72)

In [49]:
# categorize primary_use column to reduce memory on merge...

primary_use_list = building_meta_df['primary_use'].unique()
primary_use_dict = {key: value for value, key in enumerate(primary_use_list)} 
print('primary_use_dict: ', primary_use_dict)
building_meta_df['primary_use'] = building_meta_df['primary_use'].map(primary_use_dict)

gc.collect()

primary_use_dict:  {'Education': 0, 'Lodging/residential': 1, 'Office': 2, 'Entertainment/public assembly': 3, 'Other': 4, 'Retail': 5, 'Parking': 6, 'Public services': 7, 'Warehouse/storage': 8, 'Food sales and service': 9, 'Religious worship': 10, 'Healthcare': 11, 'Utility': 12, 'Technology/science': 13, 'Manufacturing/industrial': 14, 'Services': 15}


0

In [50]:
reduce_mem_usage(train_df, use_float16=True)
reduce_mem_usage(building_meta_df, use_float16=True)
reduce_mem_usage(weather_train_df, use_float16=True)

Memory usage of dataframe is 2497.05 MB
Memory usage after optimization is: 1417.31 MB
Decreased by 43.2%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 74.9%
Memory usage of dataframe is 24.53 MB
Memory usage after optimization is: 17.58 MB
Decreased by 28.3%


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3,air_temperature_mean_lag72,air_temperature_max_lag72,air_temperature_min_lag72,air_temperature_std_lag72,cloud_coverage_mean_lag72,cloud_coverage_max_lag72,cloud_coverage_min_lag72,cloud_coverage_std_lag72,dew_temperature_mean_lag72,dew_temperature_max_lag72,dew_temperature_min_lag72,dew_temperature_std_lag72,precip_depth_1_hr_mean_lag72,precip_depth_1_hr_max_lag72,precip_depth_1_hr_min_lag72,precip_depth_1_hr_std_lag72,sea_level_pressure_mean_lag72,sea_level_pressure_max_lag72,sea_level_pressure_min_lag72,sea_level_pressure_std_lag72,wind_direction_mean_lag72,wind_direction_max_lag72,wind_direction_min_lag72,wind_direction_std_lag72,wind_speed_mean_lag72,wind_speed_max_lag72,wind_speed_min_lag72,wind_speed_std_lag72
0,0,2016-01-01 00:00:00,25.000000,6.0,20.000000,-1.0,1019.5,0.0,0.000000,25.000000,25.000000,25.000000,NaN,6.000000,6.0,6.0,NaN,20.000000,20.000000,20.000000,NaN,-1.000000,-1.0,-1.0,NaN,1019.5,1019.5,1019.5,NaN,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.000000,NaN,25.000000,25.000000,25.000000,NaN,6.000000,6.0,6.0,NaN,20.000000,20.00000,20.000000,NaN,-1.000000,-1.0,-1.0,NaN,1019.5,1019.5,1019.5,NaN,0.000000,0.0,0.0,NaN,0.000000,0.000000,0.0,NaN
1,0,2016-01-01 01:00:00,24.406250,4.0,21.093750,-1.0,1020.0,70.0,1.500000,24.703125,25.000000,24.406250,0.424316,5.000000,6.0,4.0,1.414062,20.546875,21.093750,20.000000,0.777832,-1.000000,-1.0,-1.0,0.000000,1020.0,1020.0,1019.5,0.353516,35.000000,70.0,0.0,49.500000,0.750000,1.500000,0.000000,1.060547,24.703125,25.000000,24.406250,0.424316,5.000000,6.0,4.0,1.414062,20.546875,21.09375,20.000000,0.777832,-1.000000,-1.0,-1.0,0.000000,1020.0,1020.0,1019.5,0.353516,35.000000,70.0,0.0,49.50000,0.750000,1.500000,0.0,1.060547
2,0,2016-01-01 02:00:00,22.796875,2.0,21.093750,0.0,1020.0,0.0,0.000000,24.062500,25.000000,22.796875,1.137695,4.000000,6.0,2.0,2.000000,20.734375,21.093750,20.000000,0.635254,-0.666504,0.0,-1.0,0.577148,1020.0,1020.0,1019.5,0.288574,23.328125,70.0,0.0,40.406250,0.500000,1.500000,0.000000,0.866211,24.062500,25.000000,22.796875,1.137695,4.000000,6.0,2.0,2.000000,20.734375,21.09375,20.000000,0.635254,-0.666504,0.0,-1.0,0.577148,1020.0,1020.0,1019.5,0.288574,23.328125,70.0,0.0,40.40625,0.500000,1.500000,0.0,0.866211
3,0,2016-01-01 03:00:00,21.093750,2.0,20.593750,0.0,1020.0,0.0,0.000000,22.765625,24.406250,21.093750,1.650391,2.666016,4.0,2.0,1.154297,20.937500,21.093750,20.593750,0.288574,-0.333252,0.0,-1.0,0.577148,1020.0,1020.0,1020.0,0.057739,23.328125,70.0,0.0,40.406250,0.500000,1.500000,0.000000,0.866211,23.328125,25.000000,21.093750,1.750000,3.500000,6.0,2.0,1.915039,20.703125,21.09375,20.000000,0.522949,-0.500000,0.0,-1.0,0.577148,1020.0,1020.0,1019.5,0.238037,17.500000,70.0,0.0,35.00000,0.375000,1.500000,0.0,0.750000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.000000,-1.0,1020.0,250.0,2.599609,21.296875,22.796875,20.000000,1.411133,2.000000,2.0,2.0,0.000000,20.562500,21.093750,20.000000,0.550781,-0.333252,0.0,-1.0,0.577148,1020.0,1020.0,1020.0,0.099976,83.312500,250.0,0.0,144.375000,0.866699,2.599609,0.000000,1.500977,22.656250,25.000000,20.000000,2.123047,3.199219,6.0,2.0,1.789062,20.562500,21.09375,20.000000,0.550293,-0.600098,0.0,-1.0,0.547852,10

In [0]:
category_cols = ['building_id', 'site_id', 'primary_use']  # , 'meter'
feature_cols = ['square_feet', 'year_built'] + [
    'hour', 'weekend', # 'month' , 'dayofweek'
    'building_median'] + [
    'air_temperature', 'cloud_coverage',
    'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
    'wind_direction', 'wind_speed', 'air_temperature_mean_lag72',
    'air_temperature_max_lag72', 'air_temperature_min_lag72',
    'air_temperature_std_lag72', 'cloud_coverage_mean_lag72',
    'dew_temperature_mean_lag72', 'precip_depth_1_hr_mean_lag72',
    'sea_level_pressure_mean_lag72', 'wind_direction_mean_lag72',
    'wind_speed_mean_lag72', 'air_temperature_mean_lag3',
    'air_temperature_max_lag3',
    'air_temperature_min_lag3', 'cloud_coverage_mean_lag3',
    'dew_temperature_mean_lag3',
    'precip_depth_1_hr_mean_lag3', 'sea_level_pressure_mean_lag3',
    'wind_direction_mean_lag3', 'wind_speed_mean_lag3']

In [0]:
def create_X_y(train_df, target_meter):
    target_train_df = train_df[train_df['meter'] == target_meter]
    target_train_df = target_train_df.merge(building_meta_df, on='building_id', how='left')
    target_train_df = target_train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
    X_train = target_train_df[feature_cols + category_cols]
    y_train = target_train_df['meter_reading_log1p'].values

    del target_train_df
    return X_train, y_train


In [0]:
import optuna
from optuna import Trial

In [0]:
train_df_original = train_df

In [0]:
debug = False

if debug:
    # Only use 10000 data,,, for fast computation for debugging.
    train_df = train_df.sample(10000)

In [0]:
def objective(trial: Trial, fast_check=True, target_meter=0):
    folds = 5
    seed = 666
    shuffle = False
    kf = KFold(n_splits=folds, shuffle=shuffle, random_state=seed)

    X_train, y_train = create_X_y(train_df, target_meter=target_meter)
    y_valid_pred_total = np.zeros(X_train.shape[0])
    gc.collect()
    print('target_meter', target_meter, X_train.shape)

    cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
    print('cat_features', cat_features)

    models0 = []
    valid_score = 0
    for train_idx, valid_idx in kf.split(X_train, y_train):
        train_data = X_train.iloc[train_idx,:], y_train[train_idx]
        valid_data = X_train.iloc[valid_idx,:], y_train[valid_idx]

        print('train', len(train_idx), 'valid', len(valid_idx))
    #     model, y_pred_valid, log = fit_cb(train_data, valid_data, cat_features=cat_features, devices=[0,])
        model, y_pred_valid, log = fit_lgbm(trial, train_data, valid_data, cat_features=category_cols,
                                            num_rounds=1000)
        y_valid_pred_total[valid_idx] = y_pred_valid
        models0.append(model)
        gc.collect()
        valid_score += log["valid/l2"]
        if fast_check:
            break
    valid_score /= len(models0)
    return valid_score

In [0]:
# Referred https://github.com/pfnet/optuna/blob/master/examples/lightgbm_simple.py

def fit_lgbm(trial, train, val, devices=(-1,), seed=None, cat_features=None, num_rounds=1500):
    """Train Light GBM model"""
    X_train, y_train = train
    X_valid, y_valid = val
    metric = 'l2'
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'objective': 'regression',
#               'max_depth': -1,
        'learning_rate': 0.1,
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        "bagging_freq": 5,
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        "metric": metric,
        "verbosity": -1,
    }
    device = devices[0]
    if device == -1:
        # use cpu
        pass
    else:
        # use gpu
        print(f'using gpu device_id {device}...')
        params.update({'device': 'gpu', 'gpu_device_id': device})

    params['seed'] = seed

    early_stop = 20
    verbose_eval = 20

    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_features)
    watchlist = [d_train, d_valid]

    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)

    # predictions
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    
    print('best_score', model.best_score)
    log = {'train/l2': model.best_score['training']['l2'],
           'valid/l2': model.best_score['valid_1']['l2']}
    return model, y_pred_valid, log

In [59]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

target_meter 0 (5834, 34)
cat_features [31, 32, 33]
train 4667 valid 1167
training LGB:
Training until validation scores don't improve for 20 rounds.
[20]	training's l2: 0.620991	valid_1's l2: 0.610353
[40]	training's l2: 0.500895	valid_1's l2: 0.541957
Early stopping, best iteration is:
[31]	training's l2: 0.529022	valid_1's l2: 0.534991


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning:

Using categorical_feature in Dataset.

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning:

categorical_feature in param dict is overridden.



best_score defaultdict(<class 'dict'>, {'training': {'l2': 0.5290221474245208}, 'valid_1': {'l2': 0.534990983614923}})


[I 2019-10-29 08:02:28,807] Finished trial#0 resulted in value: 0.534990983614923. Current best value is 0.534990983614923 with parameters: {'num_leaves': 180, 'lambda_l1': 0.01651518673637254, 'lambda_l2': 2.932246578020604e-08, 'bagging_fraction': 0.15072508509768748, 'feature_fraction': 0.709771094138288}.


target_meter 0 (5834, 34)
cat_features [31, 32, 33]
train 4667 valid 1167
training LGB:
Training until validation scores don't improve for 20 rounds.
[20]	training's l2: 0.638582	valid_1's l2: 0.658495
[40]	training's l2: 0.450403	valid_1's l2: 0.515402
[60]	training's l2: 0.382395	valid_1's l2: 0.505138
[80]	training's l2: 0.330009	valid_1's l2: 0.505486
Early stopping, best iteration is:
[65]	training's l2: 0.369572	valid_1's l2: 0.501804
best_score defaultdict(<class 'dict'>, {'training': {'l2': 0.36957183657642967}, 'valid_1': {'l2': 0.50180369706658}})


...

[I 2019-10-29 08:02:37,044] Finished trial#7 resulted in value: 0.4814949305430575. Current best value is 0.470434066953403 with parameters: {'num_leaves': 95, 'lambda_l1': 4.078055929898415e-05, 'lambda_l2': 0.035167524223564846, 'bagging_fraction': 0.8527954141600776, 'feature_fraction': 0.7978927782576919}.


target_meter 0 (5834, 34)
cat_features [31, 32, 33]
train 4667 valid 1167
training LGB:
Training until validation scores don't improve for 20 rounds.
[20]	training's l2: 0.643846	valid_1's l2: 0.628066
[40]	training's l2: 0.522291	valid_1's l2: 0.533153
Early stopping, best iteration is:
[39]	training's l2: 0.523765	valid_1's l2: 0.532685
best_score defaultdict(<class 'dict'>, {'training': {'l2': 0.5237650153975573}, 'valid_1': {'l2': 0.5326848329655114}})


[I 2019-10-29 08:02:37,730] Finished trial#8 resulted in value: 0.5326848329655114. Current best value is 0.470434066953403 with parameters: {'num_leaves': 95, 'lambda_l1': 4.078055929898415e-05, 'lambda_l2': 0.035167524223564846, 'bagging_fraction': 0.8527954141600776, 'feature_fraction': 0.7978927782576919}.


target_meter 0 (5834, 34)
cat_features [31, 32, 33]
train 4667 valid 1167
training LGB:
Training until validation scores don't improve for 20 rounds.
[20]	training's l2: 0.563976	valid_1's l2: 0.610827
[40]	training's l2: 0.396613	valid_1's l2: 0.518888
[60]	training's l2: 0.330353	valid_1's l2: 0.519961
Early stopping, best iteration is:
[41]	training's l2: 0.392012	valid_1's l2: 0.51753
best_score defaultdict(<class 'dict'>, {'training': {'l2': 0.39201249580789127}, 'valid_1': {'l2': 0.517530179567363}})


[I 2019-10-29 08:02:38,638] Finished trial#9 resulted in value: 0.517530179567363. Current best value is 0.470434066953403 with parameters: {'num_leaves': 95, 'lambda_l1': 4.078055929898415e-05, 'lambda_l2': 0.035167524223564846, 'bagging_fraction': 0.8527954141600776, 'feature_fraction': 0.7978927782576919}.
